# Filter Change MTPtg/Rotator integration test

This notebook performs a filter change scenario integration test between the Maint Telescope pointing component (MTPtg) and the Rotator with the Camera Cable Wrap tracking the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew. 

At the end, there is a section showing how to recover the Rotator CSC from `FAULT` state. This last part can be ignored as needed. 

In [151]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

In [152]:
test_message = "Filter Change MTPtg/Rotator/CCW Integration Test"

In [153]:
d = salobj.Domain()

In [154]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")

In [155]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task)

RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteT

[None, None, None]

RemoteTelemetry(Rotator, 0, Motors) falling behind; read 34 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 12 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 38 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 38 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages


In [156]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 46 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 20 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) f

[<State.ENABLED: 2>]

RemoteEvent(MTPtg, 0, weatherDataApplied) falling behind; read 19 messages
RemoteEvent(MTPtg, 0, weatherDataApplied) falling behind; read 19 messages


In [157]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

# Setup Pointing Target

In [158]:
# Here is a small trick to get the ra before transit. Get `timeAndDate` from the pointing component, then use `RA = lst - delta`.

time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=5.)

lst = Angle(time_data.lst, unit=u.hour)
delta = Angle(10., unit=u.arcminute)
ra = lst + delta
print("=======================")
print(f"LST: {lst}")
print(f"delta: {delta}")
print(f"RA: {ra}")
print("=======================")

dec = location.lat - Angle(6., unit=u.deg)
print(f"Dec: {dec}")

target_name="Rotator test"
radec = ICRS(ra, dec)

curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print(180.-alt_az.alt.deg)

LST: 19.505805555555554 hourangle
delta: 10.0 arcmin
RA: 19.516916666666667 hourangle
Dec: -36.244728 deg
95.97255481954436


# Perform First Track

In [159]:
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"{test_message}:[START]")

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 10s")

await asyncio.sleep(10.)

print("Stop track")

await mtptg.cmd_stopTracking.start(timeout=30.)

await asyncio.sleep(0.1)

await rot.cmd_stop.start(timeout=30.)

Waiting 10s
Stop track


# Perform Filter Change

In [160]:
print("Move to 0.0 deg for filter change")

await rot.cmd_positionSet.set_start(angle=0.0)

rot.evt_inPosition.flush()

await rot.cmd_move.start(timeout=30.)

inPOS = await rot.evt_inPosition.aget()

print(inPOS.inPosition)

print("Rotator in position and waiting 120s")

await asyncio.sleep(10.)

Move to 0.0 deg for filter change
False
Rotator in position and waiting 120s


# Perform Second Track

In [161]:
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=5.)

lst = Angle(time_data.lst, unit=u.hour)
delta = Angle(10., unit=u.arcminute)
ra = lst + delta
print("=======================")
print(f"LST: {lst}")
print(f"delta: {delta}")
print(f"RA: {ra}")
print("=======================")

dec = location.lat - Angle(6., unit=u.deg)
print(f"Dec: {dec}")

target_name="Rotator test"
radec = ICRS(ra, dec)

curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print(180.-alt_az.alt.deg)

print("Start second track")

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 10s")

await asyncio.sleep(10.)

await mtptg.cmd_stopTracking.start(timeout=30.)

await asyncio.sleep(0.1)

await rot.cmd_stop.start(timeout=30.)

LST: 19.51227777777778 hourangle
delta: 10.0 arcmin
RA: 19.523388888888892 hourangle
Dec: -36.244728 deg
95.97239065282638
Start second track
Waiting 10s


Move to Zero

In [162]:
print("Move to 0.0 deg")

await rot.cmd_positionSet.set_start(angle=0.0)

rot.evt_inPosition.flush()

await rot.cmd_move.start(timeout=30.)

inPOS = await rot.evt_inPosition.aget()

print(inPOS.inPosition)

print("Test complete")

script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"{test_message}:[END]")

Move to 0.0 deg
False
Test complete


True

# Fault State Recovery

In [137]:
await rot.cmd_clearError.start(timeout=10)

RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behi

In [138]:
await asyncio.sleep(5)
#await salobj.set_summary_state(mtptg, salobj.State.STANDBY)
await salobj.set_summary_state(rot, salobj.State.STANDBY)

[<State.OFFLINE: 4>, <State.STANDBY: 5>]